In [1]:
#STIX simulator

In [2]:
import sys
sys.path.append('.')
import glob
import numpy as np
import math
from core.grid_parameters import NOMINAL as ngd


import matplotlib.pyplot as plt


from ipywidgets import interact, interact_manual, widgets


from IPython.display import clear_output

from core.stix_imager import StixImager
from core import stix_grid as sgd


%matplotlib notebook

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
def gaus2d( sun_x=0, sun_y=0, x_radius=30, y_radius=30, sigma=10, rho=0, steps=40, xmin=None, 
           xmax=None,  ymin=None, ymax=None):
    if xmin is None or ymin is None or xmax is None or ymax is None:
        xmin=sun_x-x_radius
        xmax=sun_x+x_radius
        ymin=sun_y-y_radius
        ymax=sun_y+y_radius
    xx,yy=np.meshgrid(np.linspace(xmin,xmax,steps), np.linspace(ymin,ymax,steps))
    u=(xx-sun_x)/sigma
    v=(yy-sun_y)/sigma
    c=1.-rho*rho
    z=u*u-2*rho*u*v+v*v
            # Calculating Gaussian array
    zz = np.exp(-( z/  2.0 * c ) )
    return xx,yy,zz
#x,y,z=gaus2d()

#plt.pcolormesh(x, y, z)

In [5]:
import pickle
def create_pattern_lut(det_idx):
    pkf = open(f'./data/pattern_lut_{det_idx}.pkl', 'wb')
    res=math.asin(ngd[str(det_idx+1)]['FrontPitch']/597.)*180/math.pi
    steps=2*int(1./res)
    im=StixImager(det_idx)
    xx=np.linspace(-1,1,steps)
    yy=np.linspace(-1,1,steps)    
    results={'res':res*3600,'steps':steps, 'x':xx,'y':yy, 'pattern':{}}
    print(steps)
    for i in range(steps):
        for j in range(steps):
            clear_output(wait=True)
            x=xx[i]*3600
            y=yy[j]*3600
            print(f'completed: {100*(i*steps+j)/(steps*steps):.1f}%')
          
            res=im.create(x, y, 0,0,0,0, compute_pattern=True)
            results['pattern'][(i,j)]=res
            
    pickle.dump(results, pkf)
    pkf.close()

#create_pattern_lut(25)

# Gaussian  shape source simulator

In [96]:
def get_amplitudes(pattern):
    top=pattern[0:4]
    bottom=pattern[4:8]
    
    return (0.5*(np.max(top)-np.min(top))/(np.mean(top)), 0.5*(np.max(bottom)-np.min(bottom))/(np.mean(bottom)))

In [103]:




def simulate_gaus(det_idx, sun_x, sun_y, x_radius=30, y_radius=30,sigma=10,rho=0, steps=40
            ):
 
    
    im=StixImager(det_idx)
    
    
    xx,yy,zz=gaus2d(sun_x, sun_y, x_radius, y_radius, sigma, rho, steps)
    x1d=xx.reshape(-1)
    y1d=yy.reshape(-1)
    z1d=zz.reshape(-1)
    pattern=np.zeros(12)
    
    ns=x1d.size
    for i in range(ns):
        clear_output(wait=True)
        print(f'completed: {100.*i/ns:.1f}%')
        x=x1d[i]
        y=y1d[i]
        if z1d[i]>0:
            #print(x,y)
            res=im.create(x, y, 0,0,0,0, compute_pattern=True)
            pattern+=res['pattern']*z1d[i]
    pattern=pattern/np.sum(z1d)
    result=get_amplitudes(pattern)
    res=im.create(sun_x, sun_y, 0,0,0,0, compute_pattern=True)
    fig, ((ax, ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize=(8, 8))
    ax.pcolormesh(xx, yy, zz)
    im.plot(fig, ax2, ax3)


    
    #print(pattern)
    ax3.plot(range(8),pattern[0:8], label='gaussian source')
    ax3.set_title(f'sun_x={sun_x} sun_y={sun_y} x_r={x_radius}, y_r={y_radius}, sigma={sigma} ')
    ax3.set_xlabel('Pixel')
    ax3.set_xlim(0,8)
    ax3.set_ylabel('Open area (mm)')
    ax3.legend()
    
    #result=fit_pattern(pattern)
    
    print('Pattern, result', result)
    #print(result)
    x1=np.arange(4)
    ax4.plot(x1,pattern[0:4], label='top')
    ax4.plot(x1,pattern[4:8], label='bottom')
    #x2=np.linspace(0,8,50)
    #pt=result['params'][0]
    #pb=result['params'][1]
    #print(pt)
    #y_top, y_bottom=fun_sin(x2, pt[0], pt[1], pt[2], pt[3]),fun_sin(x2, pb[0], pb[1], pb[2], pb[3])
    #ax4.plot(x2, y_top, label='top fit')
    #ax4.plot(x2, y_bottom, label='bottom fit')
    
    ax4.set_title(f'modulation: {result[0]:.2f}, {result[1]:0.2f}, sum: {np.sum(pattern[0:8]):0.2f}')
    ax4.set_xlabel('Pixel')
    ax4.set_ylabel('Open area (mm)')
    ax4.legend()
    
    
    
    plt.savefig(f'sun_x_{sun_x}_sun_y_{sun_y}_x_r_{x_radius}_y_r_{y_radius}_sigma_{sigma}.svg')
    plt.show()




In [106]:
plt.close('all')
maxPhase= 0.457628#gd['26']['FrontPitch']
interact_manual(simulate_gaus, det_idx=widgets.IntSlider(min=0,max=32,value=2,step=1),
         sun_x=widgets.IntSlider(min=-3600,max=3600,value=0, step=1),
         sun_y=widgets.IntSlider(min=-3600,max=3600,value=0,step=1),
         x_radius=widgets.IntSlider(min=0,max=340,value=100, step=1),
         y_radius=widgets.IntSlider(min=0,max=340,value=100,step=1),
            sigma=widgets.IntSlider(min=0,max=400,value=50,step=1),
                 steps=widgets.IntSlider(min=10,max=100,value=10,step=1)
        
         
        )

interactive(children=(IntSlider(value=2, description='det_idx', max=32), IntSlider(value=0, description='sun_x…

<function __main__.simulate_gaus(det_idx, sun_x, sun_y, x_radius=30, y_radius=30, sigma=10, rho=0, steps=40)>

# Double Gaussian shape source simulator

In [50]:


plt.close('all')
def simulate_double_gaus(det_idx, sun_x, sun_y, sun_x2, sun_y2, 
                         radius1=30, radius2=30,sigma1=10,sigma2=0, steps=40
            ):
 
    
    im=StixImager(det_idx)
    rho=0
    xmin=min([sun_x-radius1, sun_x2-radius2])
    xmax=max([sun_x+radius1, sun_x2+radius2])
    ymin=min([sun_y-radius1, sun_y2-radius2])
    ymax=max([sun_y+radius1, sun_y2+radius2])
    xx,yy,zz1=gaus2d(sun_x, sun_y, radius1, radius1, sigma1, rho, steps,xmin, xmax, ymin,ymax)
    _,_,zz2=gaus2d(sun_x2, sun_y2, radius2, radius2, sigma2, rho, steps,xmin, xmax,ymin, ymax)
    zz=zz1+zz2
    x1d=xx.reshape(-1)
    y1d=yy.reshape(-1)
    z1d=zz.reshape(-1)
    pattern=np.zeros(12)
    
    ns=x1d.size
    for i in range(ns):
        clear_output(wait=True)
        print(f'completed: {50.*i/ns:.1f}%')
        x=x1d[i]
        y=y1d[i]
        if z1d[i]>0:
            #print(x,y)
            res=im.create(x, y, 0,0,0,0, compute_pattern=True)
            pattern+=res['pattern']*z1d[i]
    res=im.create(sun_x, sun_y, 0,0,0,0, compute_pattern=True)
    fig, ((ax, ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize=(8, 8))
    ax.pcolormesh(xx, yy, zz)
    im.plot(fig, ax2, ax3)
    pattern=pattern/np.sum(z1d)
    ax3.plot(range(12),pattern, label='gaus source')
    #ax3.set_title(f'sun_x={sun_x} sun_y={sun_y} x_r={x_radius}, y_r={y_radius}, sigma={sigma} ')
    ax3.set_xlabel('Pixel')
    ax3.set_ylabel('Open area (mm$^2$)')
    ax3.legend()
    ax4.plot(range(12),pattern, label='gaus source')
    #ax4.set_title(f'sun_x={sun_x} sun_y={sun_y} x_r={x_radius}, y_r={y_radius}, sigma={sigma} ')
    ax4.set_xlabel('Pixel')
    ax4.set_ylabel('Open area (mm)')
    ax4.legend()
    plt.savefig(f'images/s1_{sun_x}_{sun_y}_s2_{sun_x2}_{sun_y2}_r1_{radius1}_r2_{radius2}_sigma_{sigma1}_{sigma2}.svg')
    plt.show()


#maxPhase= 0.457628#gd['26']['FrontPitch']
interact_manual(simulate_double_gaus, det_idx=widgets.IntSlider(min=0,max=32,value=19,step=1),
         sun_x=widgets.IntSlider(min=-3600,max=3600,value=0, step=1),
         sun_y=widgets.IntSlider(min=-3600,max=3600,value=0,step=1),
                         sun_x2=widgets.IntSlider(min=-3600,max=3600,value=50, step=1),
         sun_y2=widgets.IntSlider(min=-3600,max=3600,value=50,step=1),
         radius1=widgets.IntSlider(min=0,max=340,value=30, step=1),
         radius2=widgets.IntSlider(min=0,max=340,value=30,step=1),
            sigma1=widgets.IntSlider(min=0,max=200,value=10,step=1),
                sigma2=widgets.IntSlider(min=0,max=200,value=10,step=1),
                 steps=widgets.IntSlider(min=10,max=100,value=20,step=1)
        
         
        )

interactive(children=(IntSlider(value=19, description='det_idx', max=32), IntSlider(value=0, description='sun_…

<function __main__.simulate_double_gaus(det_idx, sun_x, sun_y, sun_x2, sun_y2, radius1=30, radius2=30, sigma1=10, sigma2=0, steps=40)>

In [8]:
import ipywidgets
ipywidgets.__version__

'7.6.3'